In [153]:
import camelot
import pandas as pd
import re
from MyCreds.mycreds import Capstone_AWS_PG
from sqlalchemy import create_engine

In [29]:
file = 'https://www.moe.gov.sg/microsites/psle-fsbb/assets/infographics/transition-to-secondary-school/indicative-psle-score-ranges-for-individual-secondary-schools.pdf'

table = camelot.read_pdf('data/school-directory-and-information/indicative-psle-score-ranges-for-individual-secondary-schools.pdf',pages='1-16',strip_text="\n")


In [51]:
for i in range(0,16):
    if i == 0:
        df = table[i].df.iloc[3:]
    else:
        df2 = table[i].df.iloc[3:]
        df = pd.concat([df,df2])

In [52]:
df.rename({0:'serial',
           1:"school",
           7:"score_range"},axis=1,inplace=True)

df=df[['serial','school','score_range']]

In [72]:
score_low = []
score_high = []

for i in df.iterrows():
    range = i[1][2]
    scores = re.findall(r'\d+',range)
    if len(scores) == 2:
        score_low.append(scores[0])
        score_high.append(scores[1])
    elif len(scores) == 4:
        score_low.append((int(scores[0])+int(scores[2]))/2)
        score_high.append((int(scores[1])+int(scores[3]))/2)
    elif len(scores) == 0:
        score_low.append(100)
        score_high.append(100)

In [83]:
df['score_low'] = score_low
df['score_high'] = score_high
df['average_score'] = (df['score_low'].astype(int)+df['score_high'].astype(int))/2
df['percentile_rank'] = df['average_score'].rank(pct=True, ascending=False)
df

,serial,school,score_range,score_low,score_high,average_score,percentile_rank
3,1,ADMIRALTY SECONDARY SCHOOL,15 - 20,15,20,17.5,0.102041
4,2,AHMAD IBRAHIM SECONDARY SCHOOL,10 - 17,10,17,13.5,0.482993
5,3,ANDERSON SECONDARY SCHOOL,4 - 10,4,10,7.0,0.870748
6,4,ANG MO KIO SECONDARY SCHOOL,11 - 16,11,16,13.5,0.482993
7,,5 ANGLICAN HIGH SCHOOL,5(D) - 12(P),5,12,8.5,0.782313
...,...,...,...,...,...,...,...
10,,143 YUHUA SECONDARY SCHOOL West,14 - 22,14,22,18.0,0.064626
11,144,YUSOF ISHAK SECONDARY SCHOOL#,-,100,100,100.0,0.006803
12,145,YUYING SECONDARY SCHOOL,8 - 22,8,22,15.0,0.363946
3,146,ZHENGHUA SECONDARY SCHOOL,12 - 20,12,20,16.0,0.231293


In [84]:
df.sort_values(by=['percentile_rank'])

,serial,school,score_range,score_low,score_high,average_score,percentile_rank
11,144,YUSOF ISHAK SECONDARY SCHOOL#,-,100,100,100.0,0.006803
7,63,JUNYUAN SECONDARY SCHOOL,17 - 22,17,22,19.5,0.013605
5,100,QUEENSTOWN SECONDARY SCHOOL,16 - 22,16,22,19.0,0.020408
11,56,GUANGYANG SECONDARY SCHOOL,16 - 21,16,21,18.5,0.037415
6,139,YIO CHU KANG SECONDARY SCHOOL,16 - 21,16,21,18.5,0.037415
...,...,...,...,...,...,...,...
10,8,ANGLO-CHINESE SCHOOL (INDEPENDENT) – INTEGRATE...,4 - 7,4,7,5.5,0.969388
8,33,CHIJ ST. NICHOLAS GIRLS' SCHOOL - INTEGRATED P...,4(D) - 7(M),4,7,5.5,0.969388
12,78,METHODIST GIRLS' SCHOOL (SECONDARY) - INTEGRAT...,4 - 6,4,6,5.0,0.993197
8,103,RAFFLES INSTITUTION – INTEGRATED PROGRAMME,4 - 6,4,6,5.0,0.993197


In [146]:
df['match_on'] = df['serial']+df['school']
df['match_on'] = df['match_on'].str.upper()
df['match_on'] = df['match_on'].str.replace(r'\d+','')
df['match_on'] = df['match_on'].str.replace('INTEGRATED PROGRAMME','')
df['match_on'] = df['match_on'].str.replace('#','')
df['match_on'] = df['match_on'].str.replace(' ','')
df['match_on'] = df['match_on'].str.replace('-','')
df['match_on'] = df['match_on'].str.replace('–','')
df['match_on'] = df['match_on'].str.replace('(','')
df['match_on'] = df['match_on'].str.replace(')','')
df['match_on'] = df['match_on'].str.replace('.','')
df['match_on'] = df['match_on'].str.replace("'",'')
df['match_on'] = df['match_on'].str.replace('MARISSTELLAHIGHSCHOOLNORTH','MARISSTELLAHIGHSCHOOL')
df['match_on'] = df['match_on'].str.replace('FUHUASECONDARYSCHOOLWEST','FUHUASECONDARYSCHOOL')
df['match_on'] = df['match_on'].str.replace('HUAYISECONDARYSCHOOLWEST','HUAYISECONDARYSCHOOL')
df['match_on'] = df['match_on'].str.replace('JUYINGSECONDARYSCHOOLWEST','JUYINGSECONDARYSCHOOL')
df['match_on'] = df['match_on'].str.replace('KRANJISECONDARYSCHOOLWEST','KRANJISECONDARYSCHOOL')
df['match_on'] = df['match_on'].str.replace('PEICAISECONDARYSCHOOLSOUTH','PEICAISECONDARYSCHOOL')
df['match_on'] = df['match_on'].str.replace('XINMINSECONDARYSCHOOLNORTH','XINMINSECONDARYSCHOOL')
df['match_on'] = df['match_on'].str.replace('YUHUASECONDARYSCHOOLWEST','YUHUASECONDARYSCHOOL')
df

/Users/stuartong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/stuartong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  if __name__ == '__main__':
/Users/stuartong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  # Remove the CWD from sys.path while we load stuff.
/Users/stuartong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11:

,serial,school,score_range,score_low,score_high,average_score,percentile_rank,match_on
3,1,ADMIRALTY SECONDARY SCHOOL,15 - 20,15,20,17.5,0.102041,ADMIRALTYSECONDARYSCHOOL
4,2,AHMAD IBRAHIM SECONDARY SCHOOL,10 - 17,10,17,13.5,0.482993,AHMADIBRAHIMSECONDARYSCHOOL
5,3,ANDERSON SECONDARY SCHOOL,4 - 10,4,10,7.0,0.870748,ANDERSONSECONDARYSCHOOL
6,4,ANG MO KIO SECONDARY SCHOOL,11 - 16,11,16,13.5,0.482993,ANGMOKIOSECONDARYSCHOOL
7,,5 ANGLICAN HIGH SCHOOL,5(D) - 12(P),5,12,8.5,0.782313,ANGLICANHIGHSCHOOL
...,...,...,...,...,...,...,...,...
10,,143 YUHUA SECONDARY SCHOOL West,14 - 22,14,22,18.0,0.064626,YUHUASECONDARYSCHOOL
11,144,YUSOF ISHAK SECONDARY SCHOOL#,-,100,100,100.0,0.006803,YUSOFISHAKSECONDARYSCHOOL
12,145,YUYING SECONDARY SCHOOL,8 - 22,8,22,15.0,0.363946,YUYINGSECONDARYSCHOOL
3,146,ZHENGHUA SECONDARY SCHOOL,12 - 20,12,20,16.0,0.231293,ZHENGHUASECONDARYSCHOOL


In [147]:
schools = 'data/school-directory-and-information/general-information-of-schools.csv'

all_schools = pd.read_csv(schools)

secondary = all_schools.copy()
# secondary = all_schools[all_schools['mainlevel_code']=='SECONDARY']
secondary['match_on'] = secondary['school_name']
secondary['match_on'] = secondary['match_on'].str.upper()
secondary['match_on'] = secondary['match_on'].str.replace('INTEGRATED PROGRAMME','')
secondary['match_on'] = secondary['match_on'].str.replace(r'\d+','')
secondary['match_on'] = secondary['match_on'].str.replace('#','')
secondary['match_on'] = secondary['match_on'].str.replace(' ','')
secondary['match_on'] = secondary['match_on'].str.replace('-','')
secondary['match_on'] = secondary['match_on'].str.replace('(','')
secondary['match_on'] = secondary['match_on'].str.replace(')','')
secondary['match_on'] = secondary['match_on'].str.replace('.','')
secondary['match_on'] = secondary['match_on'].str.replace("'",'')



secondary

/Users/stuartong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  # Remove the CWD from sys.path while we load stuff.
/Users/stuartong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  
/Users/stuartong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  from ipykernel import kernelapp as app
/Users/stuartong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: The default value of regex will change from True 

,school_name,url_address,address,postal_code,telephone_no,telephone_no_2,fax_no,fax_no_2,email_address,mrt_desc,...,session_code,mainlevel_code,sap_ind,autonomous_ind,gifted_ind,ip_ind,mothertongue1_code,mothertongue2_code,mothertongue3_code,match_on
0,ADMIRALTY PRIMARY SCHOOL,https://admiraltypri.moe.edu.sg/,11 WOODLANDS CIRCLE,738907,63620598,na,63627512,na,ADMIRALTY_PS@MOE.EDU.SG,Admiralty Station,...,FULL DAY,PRIMARY,No,No,No,No,Chinese,Malay,Tamil,ADMIRALTYPRIMARYSCHOOL
1,ADMIRALTY SECONDARY SCHOOL,http://www.admiraltysec.moe.edu.sg,31 WOODLANDS CRESCENT,737916,63651733,63654596,63652774,na,Admiralty_SS@moe.edu.sg,ADMIRALTY MRT,...,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil,ADMIRALTYSECONDARYSCHOOL
2,AHMAD IBRAHIM PRIMARY SCHOOL,http://www.ahmadibrahimpri.moe.edu.sg,10 YISHUN STREET 11,768643,67592906,na,67592927,na,aips@moe.edu.sg,Yishun,...,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil,AHMADIBRAHIMPRIMARYSCHOOL
3,AHMAD IBRAHIM SECONDARY SCHOOL,http://www.ahmadibrahimsec.moe.edu.sg,751 YISHUN AVENUE 7,768928,67585384,na,67557778,na,aiss@moe.edu.sg,"CANBERRA MRT, YISHUN MRT",...,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil,AHMADIBRAHIMSECONDARYSCHOOL
4,AI TONG SCHOOL,http://www.aitong.moe.edu.sg,100 Bright Hill Drive,579646,64547672,na,64532726,na,AITONG_SCH@MOE.EDU.SG,Bishan MRT,...,SINGLE SESSION,PRIMARY,Yes,No,No,No,Chinese,na,na,AITONGSCHOOL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,ZHANGDE PRIMARY SCHOOL,http://www.zhangdepri.moe.edu.sg/,51 Jalan Membina,169485,62740357,na,62763042,na,ZHANGDE_PS@MOE.EDU.SG,Tiong Bahru MRT Station,...,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil,ZHANGDEPRIMARYSCHOOL
342,ZHENGHUA PRIMARY SCHOOL,http://www.zhenghuapri.moe.edu.sg,9 Fajar Road,679002,67697478,na,67636459,na,zhenghua_ps@moe.edu.sg,LRT: Fajar Station,...,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil,ZHENGHUAPRIMARYSCHOOL
343,ZHENGHUA SECONDARY SCHOOL,http://www.zhenghuasec.moe.edu.sg,91 SENJA ROAD,677741,67639455,na,67633577,na,ZHENGHUA_SS@MOE.EDU.SG,JELAPANG LRT,...,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil,ZHENGHUASECONDARYSCHOOL
344,ZHONGHUA PRIMARY SCHOOL,http://www.zhonghuapri.moe.edu.sg,12 SERANGOON AVENUE 4,556095,62835413,na,62823882,na,zhonghua_ps@moe.edu.sg,"Ang Mo Kio MRT, Bishan MRT, Serangoon MRT",...,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil,ZHONGHUAPRIMARYSCHOOL


In [148]:
secondary[secondary['school_name'].str.contains('MARIS')]

,school_name,url_address,address,postal_code,telephone_no,telephone_no_2,fax_no,fax_no_2,email_address,mrt_desc,...,session_code,mainlevel_code,sap_ind,autonomous_ind,gifted_ind,ip_ind,mothertongue1_code,mothertongue2_code,mothertongue3_code,match_on
162,MARIS STELLA HIGH SCHOOL,http://www.marisstellahigh.moe.edu.sg,25 MOUNT VERNON ROAD,368051,62803880 (Sec),62827743 (Pri),62886905 (Sec),62827905 (Pri),msh@moe.edu.sg,BARTLEY MRT,...,SINGLE SESSION,MIXED LEVELS,Yes,Yes,No,No,Chinese,na,na,MARISSTELLAHIGHSCHOOL


In [149]:
secondary_ranked = df.merge(secondary, left_on='match_on',right_on='match_on',how='left',indicator=True)

In [150]:
secondary_ranked[secondary_ranked['_merge']!='both']

,serial,school,score_range,score_low,score_high,average_score,percentile_rank,match_on,school_name,url_address,...,session_code,mainlevel_code,sap_ind,autonomous_ind,gifted_ind,ip_ind,mothertongue1_code,mothertongue2_code,mothertongue3_code,_merge


In [140]:
df[df['school'].str.contains('MARIS')]


,serial,school,score_range,score_low,score_high,average_score,percentile_rank,match_on
7,,73 MARIS STELLA HIGH SCHOOL North,7(M) - 12,7,12,9.5,0.727891,MARISSTELLAHIGHSCHOOLNORTH


In [152]:
secondary_ranked

,serial,school,score_range,score_low,score_high,average_score,percentile_rank,match_on,school_name,url_address,...,session_code,mainlevel_code,sap_ind,autonomous_ind,gifted_ind,ip_ind,mothertongue1_code,mothertongue2_code,mothertongue3_code,_merge
0,1,ADMIRALTY SECONDARY SCHOOL,15 - 20,15,20,17.5,0.102041,ADMIRALTYSECONDARYSCHOOL,ADMIRALTY SECONDARY SCHOOL,http://www.admiraltysec.moe.edu.sg,...,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil,both
1,2,AHMAD IBRAHIM SECONDARY SCHOOL,10 - 17,10,17,13.5,0.482993,AHMADIBRAHIMSECONDARYSCHOOL,AHMAD IBRAHIM SECONDARY SCHOOL,http://www.ahmadibrahimsec.moe.edu.sg,...,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil,both
2,3,ANDERSON SECONDARY SCHOOL,4 - 10,4,10,7.0,0.870748,ANDERSONSECONDARYSCHOOL,ANDERSON SECONDARY SCHOOL,http://www.andersonsec.moe.edu.sg,...,SINGLE SESSION,SECONDARY,No,Yes,No,No,Chinese,Malay,Tamil,both
3,4,ANG MO KIO SECONDARY SCHOOL,11 - 16,11,16,13.5,0.482993,ANGMOKIOSECONDARYSCHOOL,ANG MO KIO SECONDARY SCHOOL,http://www.angmokiosec.moe.edu.sg,...,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil,both
4,,5 ANGLICAN HIGH SCHOOL,5(D) - 12(P),5,12,8.5,0.782313,ANGLICANHIGHSCHOOL,ANGLICAN HIGH SCHOOL,http://www.anglicanhigh.moe.edu.sg,...,SINGLE SESSION,SECONDARY,Yes,Yes,No,No,Chinese,na,na,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,,143 YUHUA SECONDARY SCHOOL West,14 - 22,14,22,18.0,0.064626,YUHUASECONDARYSCHOOL,YUHUA SECONDARY SCHOOL,http://www.yuhuasec.moe.edu.sg,...,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil,both
143,144,YUSOF ISHAK SECONDARY SCHOOL#,-,100,100,100.0,0.006803,YUSOFISHAKSECONDARYSCHOOL,YUSOF ISHAK SECONDARY SCHOOL,http://www.yusofishaksec.moe.edu.sg,...,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil,both
144,145,YUYING SECONDARY SCHOOL,8 - 22,8,22,15.0,0.363946,YUYINGSECONDARYSCHOOL,YUYING SECONDARY SCHOOL,http://www.yuyingsec.moe.edu.sg,...,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil,both
145,146,ZHENGHUA SECONDARY SCHOOL,12 - 20,12,20,16.0,0.231293,ZHENGHUASECONDARYSCHOOL,ZHENGHUA SECONDARY SCHOOL,http://www.zhenghuasec.moe.edu.sg,...,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil,both


In [187]:
engine = create_engine(f'postgresql+psycopg2://{Capstone_AWS_PG.username}:{Capstone_AWS_PG.password}@{Capstone_AWS_PG.host}/capstone', echo=False)

sql_query = 'select * from sg_buildings_postal_geo;'
sql_alc_cnxn = engine.connect()

postal = pd.read_sql(sql_query, sql_alc_cnxn)
postal['sch_match'] = postal['searchval']
postal['sch_match'] = postal['sch_match'].str.replace('SAINT','ST')
# postal['sch_match'] = postal['sch_match'].str.replace('GOVT.','GOVERNMENT')
postal['sch_match'] = postal['sch_match'].str.replace("(U/C)",'')
postal['sch_match'] = postal['sch_match'].str.replace("'",'')
postal['sch_match'] = postal['sch_match'].str.replace(".",'')
postal['sch_match'] = postal['sch_match'].str.replace("-",'')
postal['sch_match'] = postal['sch_match'].str.replace(' ','')
postal['sch_match'] = postal['sch_match'].str.replace('(','')
postal['sch_match'] = postal['sch_match'].str.replace(')','')
postal['sch_match'] = postal['sch_match'].str.replace('METHODISTGIRLSSCHOOL','METHODISTGIRLSSCHOOLSECONDARY')
postal['sch_match'] = postal['sch_match'].str.replace('BUKITPANJANGGOVERNMENTHIGHSCHOOL','BUKITPANJANGGOVTHIGHSCHOOL')
postal['sch_match'] = postal['sch_match'].str.replace('PAYALEBARMETHODISTGIRLSSCHOOLSECONDARYSECONDARY','PAYALEBARMETHODISTGIRLSSCHOOLSECONDARY')
postal['sch_match'] = postal['sch_match'].str.strip()
postal

/Users/stuartong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  # Remove the CWD from sys.path while we load stuff.
/Users/stuartong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  if sys.path[0] == '':
/Users/stuartong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  from ipykernel import kernelapp as app
/Users/stuartong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: The default value of regex w

,id,address,blk_no,building,latitude,longitude,longtitude,postal,road_name,short_r_name,searchval,x,y,geometry,sch_match
0,0,101A BAYFRONT AVENUE TEMPORARY SITE OFFICE SIN...,101A,TEMPORARY SITE OFFICE,1.275697,103.855652,103.855652,18895,BAYFRONT AVENUE,BAYFRONT AVE,TEMPORARY SITE OFFICE,30485.511362,28685.612665,0101000020110F0000C685CAFFC2F659404BC54D264169...,TEMPORARYSITEOFFICE
1,1,1 STRAITS BOULEVARD SINGAPORE CHINESE CULTURAL...,1,SINGAPORE CHINESE CULTURAL CENTRE,1.275829,103.849576,103.849576,18906,STRAITS BOULEVARD,STRAITS BOULEVARD,SINGAPORE CHINESE CULTURAL CENTRE,29809.365407,28700.236127,0101000020110F00008F26A5755FF6594013AF0BDFCB69...,SINGAPORECHINESECULTURALCENTRE
2,2,11A STRAITS BOULEVARD TEMPORARY SITE OFFICE SI...,11A,TEMPORARY SITE OFFICE,1.274950,103.851665,103.851665,18907,STRAITS BOULEVARD,STRAITS BOULEVARD,TEMPORARY SITE OFFICE,30041.838898,28602.987244,0101000020110F000037F1EDAE81F659407A735AA63166...,TEMPORARYSITEOFFICE
3,3,2 CENTRAL BOULEVARD IOI CENTRAL BOULEVARD TOWE...,2,IOI CENTRAL BOULEVARD TOWERS,1.279777,103.851513,103.851513,18916,CENTRAL BOULEVARD,CENTRAL BOULEVARD,IOI CENTRAL BOULEVARD TOWERS,30024.919852,29136.807026,0101000020110F0000B11B70317FF65940960CD0D6F779...,IOICENTRALBOULEVARDTOWERS
4,4,21 PARK STREET DBS MARINA BAY MRT STATION SING...,21,DBS MARINA BAY MRT STATION,1.276427,103.854598,103.854598,18925,PARK STREET,PARK ST,DBS MARINA BAY MRT STATION,30368.205612,28766.381902,0101000020110F00002399DFBAB1F65940AA1D07173F6C...,DBSMARINABAYMRTSTATION
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142881,142881,100A KRANJI LOOP SINGAPORE 887327,100A,NIL,1.433629,103.758648,103.758648,887327,KRANJI LOOP,KRANJI LOOP,100A KRANJI LOOP SINGAPORE 887327,19690.280996,46149.118386,0101000020110F0000DCF8F2AF8DF059408F5DBB7B25F0...,100AKRANJILOOPSINGAPORE887327
142882,142882,A PASIR PANJANG ROAD SINGAPORE 887328,A,NIL,1.277170,103.795840,103.795840,887328,PASIR PANJANG ROAD,PASIR PANJANG RD,A PASIR PANJANG ROAD SINGAPORE 887328,23828.902741,28848.553424,0101000020110F00007788C209EFF25940E717771F4A6F...,APASIRPANJANGROADSINGAPORE887328
142883,142883,PULAU BUKOM SINGAPORE 903808,None,NIL,1.235596,103.768645,103.768645,903808,PULAU BUKOM,PULAU BUKOM,PULAU BUKOM SINGAPORE 903808,20802.263821,24251.557450,0101000020110F000019D2057C31F15940B486DF7800C5...,PULAUBUKOMSINGAPORE903808
142884,142884,GATE C7 AIRPORT CARGO ROAD CHANGI ANIMAL AND P...,GATE C7,CHANGI ANIMAL AND PLANT QUARANTINE CENTRE,1.375285,103.996737,103.996737,918104,AIRPORT CARGO ROAD,AIRPORT CARGO RD,CHANGI ANIMAL AND PLANT QUARANTINE CENTRE,46186.488419,39698.194002,0101000020110F0000B09F7E8ACAFF59405B6A881C2B01...,CHANGIANIMALANDPLANTQUARANTINECENTRE


In [188]:
# secondary_ranked.drop(columns=['_merge'],inplace=True)
secondary_ranked_location = secondary_ranked.merge(postal, left_on='match_on',right_on='sch_match',how='left',indicator=True)

In [189]:
secondary_ranked_location[secondary_ranked_location['_merge']!='both']

,serial,school,score_range,score_low,score_high,average_score,percentile_rank,match_on,school_name,url_address,...,longtitude,postal,road_name,short_r_name,searchval,x,y,geometry,sch_match,_merge


In [190]:
postal[postal['searchval'].str.contains('METHODIST GIRLS')]

,id,address,blk_no,building,latitude,longitude,longtitude,postal,road_name,short_r_name,searchval,x,y,geometry,sch_match
84081,84081,298 LORONG AH SOO COMMIT LEARNING SCHOOLHOUSE ...,298,COMMIT LEARNING SCHOOLHOUSE @ PAYA LEBAR METHO...,1.350255,103.884847,103.884847,536741,LORONG AH SOO,LOR AH SOO,COMMIT LEARNING SCHOOLHOUSE @ PAYA LEBAR METHO...,33734.522469,36929.865687,0101000020110F00009AB7CA54A1F85940CFBD8D86A49A...,COMMITLEARNINGSCHOOLHOUSE@PAYALEBARMETHODISTGI...
84082,84082,298 LORONG AH SOO PAYA LEBAR METHODIST GIRLS' ...,298,PAYA LEBAR METHODIST GIRLS' SCHOOL (PRIMARY),1.349473,103.884971,103.884971,536741,LORONG AH SOO,LOR AH SOO,PAYA LEBAR METHODIST GIRLS' SCHOOL (PRIMARY),33748.376095,36843.418782,0101000020110F0000912CD85EA3F85940442069C07097...,PAYALEBARMETHODISTGIRLSSCHOOLSECONDARYPRIMARY
84083,84083,296 LORONG AH SOO PAYA LEBAR METHODIST GIRLS' ...,296,PAYA LEBAR METHODIST GIRLS' SCHOOL (SECONDARY),1.350968,103.885120,103.885120,536742,LORONG AH SOO,LOR AH SOO,PAYA LEBAR METHODIST GIRLS' SCHOOL (SECONDARY),33764.922058,37008.706157,0101000020110F0000CC9391CEA5F859407165C628909D...,PAYALEBARMETHODISTGIRLSSCHOOLSECONDARY
109702,109702,11 BLACKMORE DRIVE COMMIT LEARNING SCHOOHHOUSE...,11,COMMIT LEARNING SCHOOHHOUSE @ METHODIST GIRLS'...,1.332662,103.783382,103.783382,599986,BLACKMORE DRIVE,BLACKMORE DR,COMMIT LEARNING SCHOOHHOUSE @ METHODIST GIRLS'...,22442.590980,34984.581056,0101000020110F0000A2FEBBEE22F25940C88C2B889552...,COMMITLEARNINGSCHOOHHOUSE@METHODISTGIRLSSCHOOL...
109704,109704,11 BLACKMORE DRIVE METHODIST GIRLS' SCHOOL SIN...,11,METHODIST GIRLS' SCHOOL,1.332948,103.783393,103.783393,599986,BLACKMORE DRIVE,BLACKMORE DR,METHODIST GIRLS' SCHOOL,22443.804211,35016.191455,0101000020110F00005B486F1C23F25940C450B74AC153...,METHODISTGIRLSSCHOOLSECONDARY


In [191]:
secondary_ranked[secondary_ranked['school'].str.contains('METHODIST GIRLS')]

,serial,school,score_range,score_low,score_high,average_score,percentile_rank,match_on,school_name,url_address,...,nature_code,session_code,mainlevel_code,sap_ind,autonomous_ind,gifted_ind,ip_ind,mothertongue1_code,mothertongue2_code,mothertongue3_code
76,77,METHODIST GIRLS' SCHOOL (SECONDARY),6 - 8,6,8,7.0,0.870748,METHODISTGIRLSSCHOOLSECONDARY,METHODIST GIRLS' SCHOOL (SECONDARY),http://www.mgs.moe.edu.sg,...,GIRLS' SCHOOL,SINGLE SESSION,SECONDARY,No,No,No,Yes,Chinese,Malay,Tamil
77,78,METHODIST GIRLS' SCHOOL (SECONDARY) - INTEGRAT...,4 - 6,4,6,5.0,0.993197,METHODISTGIRLSSCHOOLSECONDARY,METHODIST GIRLS' SCHOOL (SECONDARY),http://www.mgs.moe.edu.sg,...,GIRLS' SCHOOL,SINGLE SESSION,SECONDARY,No,No,No,Yes,Chinese,Malay,Tamil
93,94,PAYA LEBAR METHODIST GIRLS' SCHOOL (SECONDARY),8 - 11,8,11,9.5,0.727891,PAYALEBARMETHODISTGIRLSSCHOOLSECONDARY,PAYA LEBAR METHODIST GIRLS' SCHOOL (SECONDARY),http://www.plmgss.moe.edu.sg,...,GIRLS' SCHOOL,SINGLE SESSION,SECONDARY,No,Yes,No,No,Chinese,Malay,Tamil


In [194]:
secondary_ranked_location

,serial,school,score_range,score_low,score_high,average_score,percentile_rank,match_on,school_name,url_address,...,longtitude,postal,road_name,short_r_name,searchval,x,y,geometry,sch_match,_merge
0,1,ADMIRALTY SECONDARY SCHOOL,15 - 20,15,20,17.5,0.102041,ADMIRALTYSECONDARYSCHOOL,ADMIRALTY SECONDARY SCHOOL,http://www.admiraltysec.moe.edu.sg,...,103.802398,737916,WOODLANDS CRESCENT,WOODLANDS CRES,ADMIRALTY SECONDARY SCHOOL,24559.047294,47504.787355,0101000020110F0000F32FF77D5AF35940BE1675465E22...,ADMIRALTYSECONDARYSCHOOL,both
1,2,AHMAD IBRAHIM SECONDARY SCHOOL,10 - 17,10,17,13.5,0.482993,AHMADIBRAHIMSECONDARYSCHOOL,AHMAD IBRAHIM SECONDARY SCHOOL,http://www.ahmadibrahimsec.moe.edu.sg,...,103.829719,768928,YISHUN AVENUE 7,YISHUN AVE 7,AHMAD IBRAHIM SECONDARY SCHOOL,27599.387289,46417.708755,0101000020110F0000AFB06B1C1AF559403712D3CA19FA...,AHMADIBRAHIMSECONDARYSCHOOL,both
2,3,ANDERSON SECONDARY SCHOOL,4 - 10,4,10,7.0,0.870748,ANDERSONSECONDARYSCHOOL,ANDERSON SECONDARY SCHOOL,http://www.andersonsec.moe.edu.sg,...,103.851554,569206,ANG MO KIO STREET 53,ANG MO KIO ST 53,ANDERSON SECONDARY SCHOOL,30029.389707,39593.059765,0101000020110F00006B28CCDC7FF65940E9642AF54BFD...,ANDERSONSECONDARYSCHOOL,both
3,4,ANG MO KIO SECONDARY SCHOOL,11 - 16,11,16,13.5,0.482993,ANGMOKIOSECONDARYSCHOOL,ANG MO KIO SECONDARY SCHOOL,http://www.angmokiosec.moe.edu.sg,...,103.842155,569362,ANG MO KIO STREET 22,ANG MO KIO ST 22,ANG MO KIO SECONDARY SCHOOL,28983.384761,38818.707097,0101000020110F00007085F9DDE5F55940054767DE9CE0...,ANGMOKIOSECONDARYSCHOOL,both
4,,5 ANGLICAN HIGH SCHOOL,5(D) - 12(P),5,12,8.5,0.782313,ANGLICANHIGHSCHOOL,ANGLICAN HIGH SCHOOL,http://www.anglicanhigh.moe.edu.sg,...,103.941853,487012,UPPER CHANGI ROAD,UPP CHANGI RD,ANGLICAN HIGH SCHOOL,40078.816621,34821.022739,0101000020110F00009A20C25347FC594057460888844C...,ANGLICANHIGHSCHOOL,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,,143 YUHUA SECONDARY SCHOOL West,14 - 22,14,22,18.0,0.064626,YUHUASECONDARYSCHOOL,YUHUA SECONDARY SCHOOL,http://www.yuhuasec.moe.edu.sg,...,103.722687,649406,JURONG WEST STREET 41,JURONG WEST ST 41,YUHUA SECONDARY SCHOOL,15687.948899,36590.808516,0101000020110F000092CAF68140EE5940D4C65F30138E...,YUHUASECONDARYSCHOOL,both
151,144,YUSOF ISHAK SECONDARY SCHOOL#,-,100,100,100.0,0.006803,YUSOFISHAKSECONDARYSCHOOL,YUSOF ISHAK SECONDARY SCHOOL,http://www.yusofishaksec.moe.edu.sg,...,103.896275,828676,SUMANG WALK,SUMANG WALK,YUSOF ISHAK SECONDARY SCHOOL,35006.196150,43197.558322,0101000020110F0000FFE771925CF95940C3C3C165D082...,YUSOFISHAKSECONDARYSCHOOL,both
152,145,YUYING SECONDARY SCHOOL,8 - 22,8,22,15.0,0.363946,YUYINGSECONDARYSCHOOL,YUYING SECONDARY SCHOOL,http://www.yuyingsec.moe.edu.sg,...,103.890195,538884,HOUGANG AVENUE 1,HOUGANG AVE 1,YUYING SECONDARY SCHOOL,34329.663884,37690.788632,0101000020110F0000D1416CF3F8F85940CF776730D4B6...,YUYINGSECONDARYSCHOOL,both
153,146,ZHENGHUA SECONDARY SCHOOL,12 - 20,12,20,16.0,0.231293,ZHENGHUASECONDARYSCHOOL,ZHENGHUA SECONDARY SCHOOL,http://www.zhenghuasec.moe.edu.sg,...,103.765511,677741,SENJA ROAD,SENJA RD,ZHENGHUA SECONDARY SCHOOL,20453.853994,41144.063893,0101000020110F0000AB4D5520FEF05940892BC917BF36...,ZHENGHUASECONDARYSCHOOL,both


In [195]:
secondary_ranked_location.columns.tolist()

['serial',
 'school',
 'score_range',
 'score_low',
 'score_high',
 'average_score',
 'percentile_rank',
 'match_on',
 'school_name',
 'url_address',
 'address_x',
 'postal_code',
 'telephone_no',
 'telephone_no_2',
 'fax_no',
 'fax_no_2',
 'email_address',
 'mrt_desc',
 'bus_desc',
 'principal_name',
 'first_vp_name',
 'second_vp_name',
 'third_vp_name',
 'fourth_vp_name',
 'fifth_vp_name',
 'sixth_vp_name',
 'dgp_code',
 'zone_code',
 'type_code',
 'nature_code',
 'session_code',
 'mainlevel_code',
 'sap_ind',
 'autonomous_ind',
 'gifted_ind',
 'ip_ind',
 'mothertongue1_code',
 'mothertongue2_code',
 'mothertongue3_code',
 'id',
 'address_y',
 'blk_no',
 'building',
 'latitude',
 'longitude',
 'longtitude',
 'postal',
 'road_name',
 'short_r_name',
 'searchval',
 'x',
 'y',
 'geometry',
 'sch_match',
 '_merge']

In [198]:
cols_to_use = [
 'school_name',
 'score_range',
 'score_low',
 'score_high',
 'average_score',
 'percentile_rank',
 'url_address',
 'postal_code',
 'dgp_code',
 'zone_code',
 'type_code',
 'nature_code',
 'session_code',
 'mainlevel_code',
 'sap_ind',
 'autonomous_ind',
 'gifted_ind',
 'ip_ind',
 'mothertongue1_code',
 'mothertongue2_code',
 'mothertongue3_code',
 'id',
 'address_y',
 'blk_no',
 'building',
 'latitude',
 'longitude',
 'longtitude',
 'postal',
 'road_name',
 'short_r_name',
 'searchval',
 'x',
 'y',
 'geometry',
 ]

secondary_ranked_final = secondary_ranked_location[cols_to_use]
secondary_ranked_final

,school_name,score_range,score_low,score_high,average_score,percentile_rank,url_address,postal_code,dgp_code,zone_code,...,latitude,longitude,longtitude,postal,road_name,short_r_name,searchval,x,y,geometry
0,ADMIRALTY SECONDARY SCHOOL,15 - 20,15,20,17.5,0.102041,http://www.admiraltysec.moe.edu.sg,737916,WOODLANDS,NORTH,...,1.445891,103.802398,103.802398,737916,WOODLANDS CRESCENT,WOODLANDS CRES,ADMIRALTY SECONDARY SCHOOL,24559.047294,47504.787355,0101000020110F0000F32FF77D5AF35940BE1675465E22...
1,AHMAD IBRAHIM SECONDARY SCHOOL,10 - 17,10,17,13.5,0.482993,http://www.ahmadibrahimsec.moe.edu.sg,768928,YISHUN,NORTH,...,1.436060,103.829719,103.829719,768928,YISHUN AVENUE 7,YISHUN AVE 7,AHMAD IBRAHIM SECONDARY SCHOOL,27599.387289,46417.708755,0101000020110F0000AFB06B1C1AF559403712D3CA19FA...
2,ANDERSON SECONDARY SCHOOL,4 - 10,4,10,7.0,0.870748,http://www.andersonsec.moe.edu.sg,569206,ANG MO KIO,NORTH,...,1.374340,103.851554,103.851554,569206,ANG MO KIO STREET 53,ANG MO KIO ST 53,ANDERSON SECONDARY SCHOOL,30029.389707,39593.059765,0101000020110F00006B28CCDC7FF65940E9642AF54BFD...
3,ANG MO KIO SECONDARY SCHOOL,11 - 16,11,16,13.5,0.482993,http://www.angmokiosec.moe.edu.sg,569362,ANG MO KIO,SOUTH,...,1.367337,103.842155,103.842155,569362,ANG MO KIO STREET 22,ANG MO KIO ST 22,ANG MO KIO SECONDARY SCHOOL,28983.384761,38818.707097,0101000020110F00007085F9DDE5F55940054767DE9CE0...
4,ANGLICAN HIGH SCHOOL,5(D) - 12(P),5,12,8.5,0.782313,http://www.anglicanhigh.moe.edu.sg,487012,BEDOK,EAST,...,1.331181,103.941853,103.941853,487012,UPPER CHANGI ROAD,UPP CHANGI RD,ANGLICAN HIGH SCHOOL,40078.816621,34821.022739,0101000020110F00009A20C25347FC594057460888844C...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,YUHUA SECONDARY SCHOOL,14 - 22,14,22,18.0,0.064626,http://www.yuhuasec.moe.edu.sg,649406,JURONG WEST,WEST,...,1.347186,103.722687,103.722687,649406,JURONG WEST STREET 41,JURONG WEST ST 41,YUHUA SECONDARY SCHOOL,15687.948899,36590.808516,0101000020110F000092CAF68140EE5940D4C65F30138E...
151,YUSOF ISHAK SECONDARY SCHOOL,-,100,100,100.0,0.006803,http://www.yusofishaksec.moe.edu.sg,658712,BUKIT BATOK,WEST,...,1.406937,103.896275,103.896275,828676,SUMANG WALK,SUMANG WALK,YUSOF ISHAK SECONDARY SCHOOL,35006.196150,43197.558322,0101000020110F0000FFE771925CF95940C3C3C165D082...
152,YUYING SECONDARY SCHOOL,8 - 22,8,22,15.0,0.363946,http://www.yuyingsec.moe.edu.sg,538884,HOUGANG,SOUTH,...,1.357136,103.890195,103.890195,538884,HOUGANG AVENUE 1,HOUGANG AVE 1,YUYING SECONDARY SCHOOL,34329.663884,37690.788632,0101000020110F0000D1416CF3F8F85940CF776730D4B6...
153,ZHENGHUA SECONDARY SCHOOL,12 - 20,12,20,16.0,0.231293,http://www.zhenghuasec.moe.edu.sg,677741,BUKIT PANJANG,WEST,...,1.388366,103.765511,103.765511,677741,SENJA ROAD,SENJA RD,ZHENGHUA SECONDARY SCHOOL,20453.853994,41144.063893,0101000020110F0000AB4D5520FEF05940892BC917BF36...


In [199]:
secondary_ranked_final.to_sql('secondary_schools_ranked_with_loc',sql_alc_cnxn, if_exists='replace', index=False)